In [47]:
import neuraltda.test_pipeline as ttop
import numpy as np
import neuraltda.topology as top
import h5py as h5
reload(ttop)
reload(top)

<module 'neuraltda.topology' from '/Users/brad/GentnerLab/code/NeuralTDA/neuraltda/topology.py'>

In [ ]:
block_path = '/Users/brad/experiments/B999/'
kwikfile = 'B999_S00_P00.kwik'
bin_id = '0801161542'
bin_def_file = '/Users/brad/GentnerLab/code/NeuralTDA/standard_good_binnings.txt'
n_cells = 60
maxt = 6
fs = 24000.0
dthetadt = 2*np.pi/6
kappa = 1
maxfr = 25
ncellperm = 40
nperms = 1


In [ ]:
spikes, clusters, trials = ttop.generate_test_dataset(n_cells, maxt, fs, dthetadt, kappa, maxfr)

In [ ]:
spikes

In [ ]:
popvec_fname = '/Users/brad/experiments/B999/testbin.binned'
segment_info = {'period': 1}
cluster_group = ['Good']
win_size = 50.0
top.build_population_embedding(spikes, trials, clusters, win_size, fs, cluster_group, segment_info, popvec_fname)

In [ ]:
ttop.test_pipeline(block_path, kwikfile, bin_id, bin_def_file, n_cells, maxt, fs, dthetadt, kappa, maxfr, ncellperm, nperms)

In [ ]:
import pickle


In [ ]:
pickle.load('/Volumes/gentner/btheilma/experiments/B999/topology/0801161509/B999_S00_P00-good-50ms-stim-permuted/B999_S00_P00-good-50ms-stim-permuted-0801161509-stim-test_pipeline_stimulus-bettiPersistence.pkl')

In [4]:
binned_file = '/Volumes/gentner/btheilma/experiments/B1235/phy051316/Pen02_Lft_AP200_ML800__Site03_Z3000__B1235_cat_P02_S03_1/binned_data/1608221332/B1235_cat_P02_S03_1-good-50ms-stim.binned'

In [5]:
bdf = h5.File(binned_file, 'r')

In [8]:
bds = bdf['A_scaled_burung']['0']
fs = bdf.attrs['fs']

In [43]:
np.round(6.5/0.049)

133.0

In [51]:
C = top.calc_corr_raw(bds['pop_vec'], bds['windows'], fs, 1, 4, 1)

0.049984
r_i: 23.6090225564, r_j: 1.35338345865
c_ij [  0.           0.           0.           0.           0.           0.
   3.0075188    6.01503759   3.0075188    0.           0.           3.0075188
   3.0075188    3.0075188    9.02255639   9.02255639   9.02255639
   9.02255639   9.02255639   6.01503759   3.0075188    9.02255639
   6.01503759   9.02255639  15.03759398  12.03007519   9.02255639
   9.02255639  12.03007519  12.03007519  12.03007519  15.03759398
  15.03759398  12.03007519   9.02255639   9.02255639  12.03007519
  12.03007519   9.02255639   9.02255639   6.01503759   3.0075188
   6.01503759   6.01503759   3.0075188    3.0075188    3.0075188    0.           0.
   3.0075188    3.0075188    3.0075188    3.0075188    3.0075188
   6.01503759   6.01503759   6.01503759   3.0075188    3.0075188
   6.01503759   3.0075188    0.           0.           6.01503759
   6.01503759   9.02255639  18.04511278  18.04511278  15.03759398
  15.03759398  18.04511278   9.02255639   6.01503759  15.

In [52]:
C

0.11295116772823777

In [13]:
wins = bds['windows']

In [14]:
wins

<HDF5 dataset "windows": shape (133, 2), type "<f8">

In [15]:
wins[0, 1]

5669157.0

In [16]:
wins[0, 0]

5667595.0

In [18]:
wins[0,1]-wins[0,0]

1562.0

In [19]:
fs

31250.0